In [2]:
import numpy as np
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, filename):
        """
        Initialize the dataset by loading and preprocessing the data.
        """
        super(CustomDataset, self).__init__()
        self.data = []
        self.labels = []
        self._load_data(filename)

    def _load_data(self, filename):
        """
        Load and preprocess the data from a text file.
        """
        with open(filename, 'r') as f:
            for line in f:
                inp, outp = line.strip().split('output')
                inp = list(map(float, inp.strip().split(' ')))
                outp = list(map(int, outp.strip().split(' ')))


                reordered_inp = inp[0:2] + inp[2:4] + inp[4:6] + inp[6:8] + inp[8:10]
                features = np.array(reordered_inp, dtype=np.float32)

                label = 1 if 1 in outp else 0

                self.data.append(features)
                self.labels.append(label)

    def __len__(self):
        """
        Return the number of samples in the dataset.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Return a single data sample and its label.
        """
        return self.data[idx], self.labels[idx]


filename = "/content/convex_hull_5_test.txt"
dataset = CustomDataset(filename)


batch_size = 16
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


for batch_data, batch_labels in train_loader:
    print("Batch Data Shape:", batch_data.shape)
    print("Batch Labels Shape:", batch_labels.shape)
    break


Batch Data Shape: torch.Size([16, 10])
Batch Labels Shape: torch.Size([16])


In [3]:
print(dataset[0])

(array([0.99624294, 0.928026  , 0.24557766, 0.80411434, 0.43667576,
       0.3330164 , 0.36675233, 0.7146409 , 0.8069548 , 0.14914954],
      dtype=float32), 1)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim


class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x


input_size = 10
model = SimpleNN(input_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def train_model(train_loader, model, criterion, optimizer, epochs=400):
    for epoch in range(400):
        epoch_loss = 0.0
        for data, labels in train_loader:

            data = data.float()
            labels = labels.float().unsqueeze(1)


            optimizer.zero_grad()


            outputs = model(data)


            loss = criterion(outputs, labels)


            loss.backward()
            optimizer.step()


            epoch_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

train_model(train_loader, model, criterion, optimizer, epochs=400)




Epoch 1/400, Loss: 0.3248322846353054
Epoch 2/400, Loss: 0.2901763749837875
Epoch 3/400, Loss: 0.24411755818128586
Epoch 4/400, Loss: 0.19679702846854924
Epoch 5/400, Loss: 0.18203064136654137
Epoch 6/400, Loss: 0.17454003571271898
Epoch 7/400, Loss: 0.1726642973482609
Epoch 8/400, Loss: 0.16719946441948413
Epoch 9/400, Loss: 0.16515434776693583
Epoch 10/400, Loss: 0.16004037075638772
Epoch 11/400, Loss: 0.15985282829031347
Epoch 12/400, Loss: 0.1565074865952134
Epoch 13/400, Loss: 0.15507349071353674
Epoch 14/400, Loss: 0.15185764811933042
Epoch 15/400, Loss: 0.14977611453682185
Epoch 16/400, Loss: 0.14848866698145866
Epoch 17/400, Loss: 0.14748157742246987
Epoch 18/400, Loss: 0.14626728284582496
Epoch 19/400, Loss: 0.14567480903193356
Epoch 20/400, Loss: 0.14298022611364722
Epoch 21/400, Loss: 0.14346357369869947
Epoch 22/400, Loss: 0.141773460232839
Epoch 23/400, Loss: 0.1408630154170096
Epoch 24/400, Loss: 0.14180102859959007
Epoch 25/400, Loss: 0.13966169677004217
Epoch 26/400, Lo

In [5]:
def save_model(model, path="trained_model.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

save_model(model)

Model saved to trained_model.pth


In [6]:
class CustomDatasetTest(Dataset):
    def __init__(self, filename):
        """
        Initialize the dataset by loading and preprocessing the data.
        """
        super(CustomDatasetTest, self).__init__()
        self.data = []
        self.labels = []
        self._load_data(filename)

    def _load_data(self, filename):
        """
        Load and preprocess the data from a text file.
        """
        with open(filename, 'r') as f:
            for line in f:

                inp, outp = line.strip().split('output')
                inp = list(map(float, inp.strip().split(' ')))
                outp = list(map(int, outp.strip().split(' ')))
                for i in range(0,5):
                  reordered_inp=inp[i*2:i*2+2]+inp[i*2+2:10]+inp[0:i*2]
                  features = np.array(reordered_inp, dtype=np.float32)

                  self.data.append(features)
                  if i+1 in outp:
                    self.labels.append(1)
                  else:
                    self.labels.append(0)


    def __len__(self):
        """
        Return the number of samples in the dataset.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Return a single data sample and its label.
        """
        return self.data[idx], self.labels[idx]


filename = "/content/convex_hull_5_test.txt"
dataset_test = CustomDatasetTest(filename)


batch_size = 5
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [7]:
for data,labels in test_loader:
  print(data)
  print(labels)
  break

tensor([[0.9962, 0.9280, 0.2456, 0.8041, 0.4367, 0.3330, 0.3668, 0.7146, 0.8070,
         0.1491],
        [0.2456, 0.8041, 0.4367, 0.3330, 0.3668, 0.7146, 0.8070, 0.1491, 0.9962,
         0.9280],
        [0.4367, 0.3330, 0.3668, 0.7146, 0.8070, 0.1491, 0.9962, 0.9280, 0.2456,
         0.8041],
        [0.3668, 0.7146, 0.8070, 0.1491, 0.9962, 0.9280, 0.2456, 0.8041, 0.4367,
         0.3330],
        [0.8070, 0.1491, 0.9962, 0.9280, 0.2456, 0.8041, 0.4367, 0.3330, 0.3668,
         0.7146]])
tensor([1, 1, 1, 0, 1])


In [20]:
for data,labels in test_loader:
  print(data[0][0])
  print(labels)
  break

tensor(0.9962)
tensor([1, 1, 1, 0, 1])


In [8]:
num=0
dict_test={}
dict_data={}
for data,labels in test_loader:
  data = data.float()
  labels = labels.float().unsqueeze(1)
  with torch.no_grad():
    outputs = model(data)
  predictions = (outputs >= 0.5).float()
  dict_test[num]=[]
  dict_data[num]=[]
  for i in range(0,5):
    if predictions[i]==1:
      dict_test[num].append([data[i][0],data[i][1]])
    if labels[i]==1:
      dict_data[num].append([data[i][0],data[i][1]])
  num+=1



In [9]:
print(dict_test[0])

[[tensor(0.9962), tensor(0.9280)], [tensor(0.2456), tensor(0.8041)], [tensor(0.4367), tensor(0.3330)], [tensor(0.8070), tensor(0.1491)]]


In [10]:
for keys in dict_test:
  nested_float_list = [[float(value) for value in row] for row in dict_test[keys]]
  dict_test[keys] = np.array(nested_float_list, dtype=np.float32)

In [14]:
print(dict_test[0])
centroid=[]
for keys in dict_test:
  centroid.append(dict_test[keys].mean(axis=0))

[[0.99624294 0.928026  ]
 [0.24557766 0.80411434]
 [0.43667576 0.3330164 ]
 [0.8069548  0.14914954]]


In [20]:
data=[]
dict_ordered={}
for keys in dict_test:
  centroid=dict_test[keys].mean(axis=0)
  angles = np.arctan2(dict_test[keys][:, 1] - centroid[1], dict_test[keys][:, 0] - centroid[0])
  sorted_indices = np.argsort(angles)
  sorted_points = dict_test[keys][sorted_indices]
  dict_ordered[keys]=sorted_points
  data.append((dict_test[keys],sorted_points))


In [12]:
!pip install tensorboardX

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.5 MB/s eta 0:00:00


In [33]:
import numpy as np
import tqdm
import torch
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_
import argparse
import logging
import sys
from tensorboardX import SummaryWriter
from types import SimpleNamespace


def generate_data(num_samples, max_seq_len=5):
    inputs, outputs = [], []
    for _ in range(num_samples):
        num_points = np.random.randint(3, max_seq_len + 1)
        points = np.random.rand(num_points, 2)  # Random 2D points
        centroid = points.mean(axis=0)
        angles = np.arctan2(points[:, 1] - centroid[1], points[:, 0] - centroid[0])
        sorted_indices = np.argsort(angles)
        sorted_points = points[sorted_indices]
        inputs.append(points)
        outputs.append(sorted_points)
    return inputs, outputs



class ConvexHullDataset(Dataset):
    def __init__(self, data, max_seq_len=5):
        self.data = data
        self.max_seq_len = max_seq_len
        self.pad_token = np.array([0, 0], dtype=np.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        inp, out = self.data[index]


        if len(inp) == 0 or len(out) == 0:
            raise ValueError(f"Invalid data at index {index}: input or output is empty.")

        inp_len = len(inp)
        out_len = len(out)


        padded_inp = np.vstack([inp, np.tile(self.pad_token, (self.max_seq_len - inp_len, 1))])
        padded_inp = padded_inp[:self.max_seq_len]  # Truncate if needed


        padded_out = np.vstack([out, np.tile(self.pad_token, (self.max_seq_len - out_len, 1))])
        padded_out = padded_out[:self.max_seq_len]  # Truncate if needed

        return (
            torch.tensor(padded_inp, dtype=torch.float32),
            torch.tensor(padded_out, dtype=torch.float32),
        )


class PointerNet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(PointerNet, self).__init__()
        self.hidden_dim = hidden_dim


        self.input_projection = torch.nn.Linear(input_dim, hidden_dim)


        self.encoder = torch.nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.decoder = torch.nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)


        self.attention = torch.nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = torch.nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        """
        :param x: Input tensor of shape (batch_size, seq_len, input_dim)
        :return: Tensor of predicted coordinates (batch_size, seq_len, input_dim)
        """

        x = self.input_projection(x)


        encoder_out, _ = self.encoder(x)


        decoder_input = encoder_out[:, 0:1, :]
        decoder_states = None
        outputs = []

        for _ in range(encoder_out.size(1)):

            decoder_out, decoder_states = self.decoder(decoder_input, decoder_states)
            attention_scores = torch.tanh(self.attention(decoder_out))
            context_vector = torch.sum(attention_scores * encoder_out, dim=1, keepdim=True)
            predicted_coord = self.output_layer(context_vector)


            outputs.append(predicted_coord)


            decoder_input = self.input_projection(predicted_coord)


        outputs = torch.cat(outputs, dim=1)
        return outputs
class PointerNetLoss(nn.Module):
    def __init__(self):
        super(PointerNetLoss, self).__init__()
        self.mse_loss = torch.nn.MSELoss()

    def forward(self, predicted, target):
        """
        :param predicted: Predicted coordinates of shape (batch_size, seq_len, input_dim)
        :param target: Ground truth coordinates of shape (batch_size, seq_len, input_dim)
        :return: Scalar loss value
        """
        return self.mse_loss(predicted, target)

In [34]:
if __name__ == "__main__":

    args = SimpleNamespace(
        gpu=0,
        bz=32,
        max_in_seq_len=5,
        max_out_seq_len=5,
        rnn_hidden_size=128,
        num_layers=1,
        lr=1e-3,
        clip_norm=5.0,
        weight_decay=0.1,
        check_interval=20,
        nepoch=100,
        log_dir="./log",
    )


    device = torch.device("cuda" if args.gpu >= 0 and torch.cuda.is_available() else "cpu")


    logger = logging.getLogger("PointerNet Evaluation")
    formatter = logging.Formatter('%(asctime)s %(levelname)-8s: %(message)s')
    console_handler = logging.StreamHandler()
    console_handler.formatter = formatter
    logger.addHandler(console_handler)
    logger.setLevel(logging.DEBUG)


    model = PointerNet(
        input_dim=2,
        hidden_dim=args.rnn_hidden_size,
        num_layers=args.num_layers
    ).to(device)

    model.load_state_dict(torch.load("/content/pointer_net_model (2).pth", map_location=device))
    model.eval()
    logger.info("Model loaded and ready for inference.")





<ipython-input-34-3b950b332261>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/pointer_net_model (2).pth", map_location=device)

In [26]:
test_ds = ConvexHullDataset(data, args.max_in_seq_len)
test_dl = DataLoader(test_ds, batch_size=1, shuffle=False)

In [27]:
print(len(test_dl))

10000


In [28]:

dict_predicted={}
import logging


logger = logging.getLogger("PointerNet Evaluation")
if not logger.handlers:
    formatter = logging.Formatter('%(asctime)s %(levelname)-8s: %(message)s')
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    logger.setLevel(logging.DEBUG)
nu=0

for idx, (inp, target) in enumerate(test_dl):
    inp = inp.to(device)
    target = target.to(device)

    with torch.no_grad():
        logits = model(inp)
        predicted = logits.cpu().numpy().squeeze()
    inp_points = inp.cpu().numpy().squeeze()
    target_points = target.cpu().numpy().squeeze()
    dict_predicted[nu]=predicted
    nu+=1

In [32]:
print(dict_predicted[9999])
print(dict_ordered[9999])


[[0.39127403 0.22544329]
 [0.67497987 0.11289382]
 [0.8686179  0.12667394]
 [0.7766644  0.57782996]
 [0.49923903 0.79638934]]
[[0.40397987 0.2789664 ]
 [0.58302546 0.05963871]
 [0.8895179  0.02790996]
 [0.8291433  0.43560365]
 [0.61540806 0.9145103 ]]


In [8]:

model.eval()


correct_predictions = 0
total_predictions = 0


for data, labels in test_loader:

    data = data.float()
    labels = labels.float().unsqueeze(1)


    with torch.no_grad():
        outputs = model(data)


    predictions = (outputs >= 0.5).float()


    correct_predictions += (predictions == labels).sum().item()
    total_predictions += labels.size(0)


accuracy = (correct_predictions / total_predictions) * 100
print(f"Accuracy over the entire dataset: {accuracy:.2f}%")





Accuracy over the entire dataset: 94.51%
